## I Know OpenCV and know to uses but Took Help of Chat Gpt bCoz data complexity

In [14]:
import cv2
import os
import random
import shutil

# --------- Step 1: Set paths ----------
source_paths = [
    "Celeb-DF/celeb-real",
    "Celeb-DF/celeb-synthesis",
    "Celeb-DF/YouTube-real",
    "Celeb-DF-v2/celeb-real",
    "Celeb-DF-v2/celeb-synthesis",
    "Celeb-DF-v2/YouTube-real"
]


output_base = "extracted_frames"
train_ratio = 0.8

# --------- Step 2: Create Output Folders ----------
for split in ["train", "val"]:
    for label in ["real", "fake"]:
        os.makedirs(os.path.join(output_base, split, label), exist_ok=True)

# --------- Step 3: Function to extract frames ----------
def extract_frames(video_path, output_folder, frame_rate=10):
    cap = cv2.VideoCapture(video_path)
    count = 0
    saved = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_frame{count}.jpg"
            cv2.imwrite(os.path.join(output_folder, filename), frame)
            saved += 1
        count += 1
    cap.release()
    return saved

# --------- Step 4: Process each video ----------
for path in source_paths:
    label = "real" if "real" in path else "fake"
    videos = os.listdir(path)
    random.shuffle(videos)
    
    split_index = int(len(videos) * train_ratio)
    train_videos = videos[:split_index]
    val_videos = videos[split_index:]

    for split, split_videos in zip(["train", "val"], [train_videos, val_videos]):
        for video in split_videos:
            video_path = os.path.join(path, video)
            if not video_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
                continue  # skip non-video files
            output_folder = os.path.join(output_base, split, label)
            try:
                extract_frames(video_path, output_folder)
            except:
                print(f"⚠️ Error reading: {video_path}")


## Using CNN


##### Importing libraries

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### Image size and path

In [19]:
img_size = (96, 96)
batch_size = 32

##### Training the Data into Train and val(test)

In [20]:

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    shear_range=0.1,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "extracted_frames/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    "extracted_frames/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset='validation'
)

Found 212700 images belonging to 2 classes.
Found 53174 images belonging to 2 classes.


#### CNN Model Training

In [21]:

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(*img_size, 3)),
    MaxPooling2D(),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

#### Compiling the data after Using CNN 

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the Frames which we exctracted

In [24]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1895s 285ms/step - accuracy: 0.7821 - loss: 0.5211 - val_accuracy: 0.7755 - val_loss: 0.5514
Epoch 2/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 2096s 315ms/step - accuracy: 0.7825 - loss: 0.5120 - val_accuracy: 0.7685 - val_loss: 0.5763
Epoch 3/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1708s 257ms/step - accuracy: 0.7851 - loss: 0.5012 - val_accuracy: 0.7613 - val_loss: 0.6343
Epoch 4/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1735s 261ms/step - accuracy: 0.7854 - loss: 0.4970 - val_accuracy: 0.7560 - val_loss: 0.6137
Epoch 5/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1616s 243ms/step - accuracy: 0.7846 - loss: 0.4930 - val_accuracy: 0.7360 - val_loss: 0.7242
Epoch 6/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1669s 251ms/step - accuracy: 0.7840 - loss: 0.4936 - val_accuracy: 0.7401 - val_loss: 0.7518
Epoch 7/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1871s 282ms/step - accuracy: 0.7866 - loss: 0.4875 - val_accuracy: 0.7305 - val_loss: 0.6843
Epoch 8/10
6647/6647 ━━━━━━━━━━━━━━━━━━━━ 1960s 295ms/s

## Predicting on Test case(val)

In [26]:
import numpy as np
val_generator.reset()
predictions = model.predict(val_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = val_generator.classes

1662/1662 ━━━━━━━━━━━━━━━━━━━━ 258s 155ms/step


# Accuracy Score and Confusion Matrix With Sklearn

In [28]:
from sklearn.metrics import accuracy_score


accuracy_score(y_true , y_pred)

0.7835032158573739

In [30]:
model.save('deepfake_detection_model.h5')